In [1]:
pip install dictances

You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
from __future__ import division
import json
import numpy as np
from decimal import Decimal
from numpy import expand_dims, array, exp, max
import scipy
from scipy.special import rel_entr
import csv
import statistics
from numpy import expand_dims, array, exp, max
import math
from scipy.spatial import distance
from scipy.stats import wasserstein_distance

In [3]:
from numpy.linalg import norm
from dictances import bhattacharyya, bhattacharyya_coefficient

In [4]:
# import the timeit module
import timeit
import math
import tracemalloc

In [5]:
def hellinger_fast(p, q):
    """Hellinger distance between two discrete distributions.
       In pure Python.
       Fastest version.
    """
    return sum([ (math.sqrt(p_i) - math.sqrt(q_i))**2 for p_i, q_i in zip(p,q) ])

In [6]:
# to suppress division errors
np.seterr(divide='ignore', invalid='ignore')
new_final_source_array = []

In [7]:
# Opening final JSON files
contrast_source_data = open('../../../files/mobilenetv2/glcm/source_domain_data_new_correlation_glcm.json')

#for target data
contrast_target_data = open('../../../files/mobilenetv2/glcm/stanford/stanford_domain_data_new_correlation_glcm.json')

In [8]:
# returns JSON object
contrast_source_data_d = json.load(contrast_source_data)

contrast_target_data_d = json.load(contrast_target_data)

In [9]:
# kdl dictionary
kdl_dictionary = {}

In [10]:
def KL(p, q):
    a = []
    b = []
    for k in p:
        a.append(Decimal(k))

    for l in q:
        b.append(Decimal(l))

    a = np.asarray(a, dtype=np.float64)
    b = np.asarray(b, dtype=np.float64)

    return np.sum(np.where(((a != 0) & (b != 0)), a * np.log(a / b), 0))

In [11]:
def softMax(temperature, features_list):
    feat_list = [x / temperature for x in features_list]
    feature_list = exp(feat_list - max(feat_list))
    return feature_list / feature_list.sum(axis=0)

In [12]:
# get the data for the contrast target image
contrast_target_data_array = []
for p in range(0, len(contrast_target_data_d['target_images'])):
    for i, j in contrast_target_data_d['target_images'][p].items():
        contrast_target_data_array.append((i,np.array(j)))
contrast_target_data.close()

In [13]:
print(len(contrast_target_data_array))

1568


In [33]:
###contrast comparison code
contrast_All_Images = []
contrast_average_dkl = []

#check the timer_start
start_bhattacharyya = timeit.default_timer()
#check memory consumption
# starting the monitoring
tracemalloc.start()

for x in contrast_target_data_array[:2]:
#for x in target_data_array:
    t_images_btween_0_05 = []
    t_images_btween_05_1 = []
    t_images_btween_1_2 = []
    t_images_grt_2_5 = []
    t_images_grt_5_10 = []
    t_images_grt_10 = []
    
    image_name_class =  x[0].split('_/') #name[0]; class[1]
    compare_min_max = []
    
    
            
    # get the data for the source images
    for m in range(0, len(contrast_source_data_d['source_images'])):
        #print("source",source_data_d['source_images'][m][0])
        for k, v in contrast_source_data_d['source_images'][m].items():
            # compare the value of v with that of the target image
            # if 'caterpillar' in k.lower():
            # remove the zeros from the list
            new_v = np.array(v).flatten()
            #get the top items in x
            items = np.sort(x[1])
            item_size = items[-len(new_v):]
            new_x = item_size.flatten()
            
            new_d = softMax(0.5, new_x)     
            new_y = softMax(0.5, new_v)
            
            kdl = rel_entr( new_x, new_v, out=None)
            new_bhattacharyya = bhattacharyya(dict(enumerate(new_x.flatten(), 1)),dict(enumerate(new_v.flatten(), 1)) )
            new_wasserstein = wasserstein_distance(new_x,new_v)
            new_jensen = distance.jensenshannon(new_x,new_v)
            new_hellinger = hellinger_fast(new_x,new_v)
            
            compare_min_max.append((image_name_class[0],image_name_class[1],k, max(new_bhattacharyya)))
            
contrast_All_Images.append(compare_min_max)
    
contrast_source_data.close()

#stop the execution
stop_bhattacharyya = timeit.default_timer()
execution_time = stop_bhattacharyya - start_bhattacharyya
print(" execution time:-", execution_time)

# displaying the memory
print(" memory",tracemalloc.get_traced_memory()[1]-tracemalloc.get_traced_memory()[0])
 
# stopping the library
tracemalloc.stop()


 execution time:- 7.157638385891914
 memory 12262
